In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matsim
import importlib
import pollutants
import matplotlib.pyplot as plt
from lxml import etree

In [ ]:
importlib.reload(pollutants)


# Path

In [2]:
root_path = r'../../../../data/intermediate/test/freightEmissions/'
iter_kw = "0"
basic_scenario_outputs = 'scenarioBasic/iter'+iter_kw+'/outputs/'
van_scenario_outputs = 'scenarioVan/iter'+iter_kw+'/outputs/'
cargo_bike_scenario_outputs = 'scenarioCB/iter' + iter_kw +'/outputs/'

scenario_kw = van_scenario_outputs


# Read Network

In [3]:
network = matsim.read_network(
    root_path + 'GemeenteLeuvenWithHbefaType.xml.gz'
)
network_gdf = network.as_geo()
network_gdf

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,33.333333,2666.666667,3.5,1,"bike,car",100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,33.333333,2666.666667,3.5,1,"bike,car",100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
103748,183.145612,33.333333,2666.666667,3.5,1,car,splitLink_9904390_5_2,splitNode_9904390_5_1,19792038,"LINESTRING (175632.401 172452.198, 175505.340 ..."
103749,102.305105,33.333333,2666.666667,3.5,1,car,splitLink_9904410_0_1,80702464,splitNode_9904410_0_1,"LINESTRING (176002.946 172889.690, 176056.498 ..."
103750,102.305105,33.333333,2666.666667,3.5,1,car,splitLink_9904410_0_2,splitNode_9904410_0_1,100399122,"LINESTRING (176056.498 172976.855, 176110.050 ..."
103751,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


In [5]:
nodes = network.nodes
nodes

,x,y,node_id
0,175270.308015,173512.798102,1000630419
1,175160.942490,173472.188651,1000630421
2,175238.142000,173547.294517,1000630459
3,175338.370411,173481.813410,1000630462
4,175180.404081,173465.163020,1000630464
...,...,...,...
59045,174985.241824,172934.604012,splitNode_9904064_1_1
59046,174985.241824,172934.604012,splitNode_9904064_r_1_1
59047,175632.401153,172452.198438,splitNode_9904390_5_1
59048,176056.498044,172976.854738,splitNode_9904410_0_1


# Read carrier plans

In [22]:
# Load the XML file
file_path = root_path + scenario_kw + 'output_carriers.xml.gz'
tree = etree.parse(file_path)
root = tree.getroot()

# Namespace to handle the XML structure
namespace = {"ns": "http://www.matsim.org/files/dtd"}

In [19]:
# Function to extract shipments
def extract_shipments(root, namespace):
    shipments = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        for shipment in carrier.find("ns:shipments", namespaces=namespace).findall("ns:shipment", namespaces=namespace):
            shipments.append({
                "carrier_id": carrier_id,
                "shipment_id": shipment.get("id"),
                "from": shipment.get("from"),
                "to": shipment.get("to"),
                "size": shipment.get("size"),
                "start_pickup": shipment.get("startPickup"),
                "end_pickup": shipment.get("endPickup"),
                "start_delivery": shipment.get("startDelivery"),
                "end_delivery": shipment.get("endDelivery"),
                "pickup_service_time": shipment.get("pickupServiceTime"),
                "delivery_service_time": shipment.get("deliveryServiceTime")
            })
    return pd.DataFrame(shipments)

In [24]:
# Function to extract vehicles
def extract_vehicles(root, namespace):
    vehicles = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        capabilities = carrier.find("ns:capabilities", namespaces=namespace)
        if capabilities is not None:
            for vehicle in capabilities.find("ns:vehicles", namespaces=namespace).findall("ns:vehicle", namespaces=namespace):
                vehicles.append({
                    "carrier_id": carrier_id,
                    "vehicle_id": vehicle.get("id"),
                    "depot_link_id": vehicle.get("depotLinkId"),
                    "type_id": vehicle.get("typeId"),
                    "earliest_start": vehicle.get("earliestStart"),
                    "latest_end": vehicle.get("latestEnd")
                })
    return pd.DataFrame(vehicles)

In [45]:
def extract_paired_tours_from_file(root, namespace):
    tours = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        plans = carrier.find("ns:plans", namespaces=namespace)
        if plans is not None:
            for plan in plans.findall("ns:plan", namespaces=namespace):
                for tour in plan.findall("ns:tour", namespaces=namespace):
                    tour_id = tour.get("tourId")
                    vehicle_id = tour.get("vehicleId")
                    
                    # Pairing `act` and `leg`
                    acts = tour.findall("ns:act", namespaces=namespace)
                    legs = tour.findall("ns:leg", namespaces=namespace)
                    
                    # Ensure we pair correctly by iterating over both
                    for act, leg in zip(acts, legs):
                        tours.append({
                            "carrier_id": carrier_id,
                            "plan_score": plan.get("score"),
                            "plan_selected": plan.get("selected"),
                            "tour_id": tour_id,
                            "vehicle_id": vehicle_id,
                            "activity_type": act.get("type"),
                            "shipment_id": act.get("shipmentId"),
                            "activity_end_time": act.get("end_time"),
                            "leg_dep_time": leg.get("expected_dep_time"),
                            "leg_transp_time": leg.get("expected_transp_time"),
                            "route": leg.find("ns:route", namespaces=namespace).text
                        })
    tours_df = pd.DataFrame(tours)
    tours_df['route'] = tours_df['route'].str.split(' ')
    return tours_df.query("plan_selected=='true'")

In [23]:
shipments_df = extract_shipments(root, namespace)
shipments_df

,carrier_id,shipment_id,from,to,size,start_pickup,end_pickup,start_delivery,end_delivery,pickup_service_time,delivery_service_time
0,carrier_0,shipment_77,333784188_r_3,147856946_r_3,15,05:06:00,05:56:00,06:06:00,08:06:00,00:00:00,00:01:30
1,carrier_0,shipment_78,333784188_r_3,891578009_r_0,25,17:00:00,17:50:00,18:00:00,20:00:00,00:00:00,00:01:30
2,carrier_0,shipment_79,333784188_r_3,10269420_1,16,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
3,carrier_0,shipment_73,27566523_11,722950838_r_0,14,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30
4,carrier_0,shipment_74,333784188_r_3,126348346_10,11,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
...,...,...,...,...,...,...,...,...,...,...,...
195,carrier_0,shipment_134,27566523_11,893949566_0,10,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30
196,carrier_0,shipment_133,27566523_11,198882510_9-198882510_10,9,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
197,carrier_0,shipment_132,333784188_r_3,95890227_5,20,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
198,carrier_0,shipment_131,27566523_11,372559055_1,12,05:06:00,05:56:00,06:06:00,08:06:00,00:00:00,00:01:30


In [25]:
vehicles_df = extract_vehicles(root, namespace)
vehicles_df

,carrier_id,vehicle_id,depot_link_id,type_id,earliest_start,latest_end
0,carrier_0,carrier0_Van_0,27566523_11,Van,00:00:00,596523:14:07
1,carrier_0,carrier0_Van_1,333784188_r_3,Van,00:00:00,596523:14:07


In [46]:
tour_df = extract_paired_tours_from_file(root, namespace)
tour_df

,carrier_id,plan_score,plan_selected,tour_id,vehicle_id,activity_type,shipment_id,activity_end_time,leg_dep_time,leg_transp_time,route
804,carrier_0,-2997.1683157259895,true,1,carrier0_Van_1,start,None,00:00:00,00:00:00,00:00:00,None
805,carrier_0,-2997.1683157259895,true,1,carrier0_Van_1,pickup,shipment_16,None,05:06:00,00:00:00,None
806,carrier_0,-2997.1683157259895,true,1,carrier0_Van_1,pickup,shipment_3,None,05:06:00,00:00:00,None
807,carrier_0,-2997.1683157259895,true,1,carrier0_Van_1,pickup,shipment_185,None,05:06:00,00:00:00,None
808,carrier_0,-2997.1683157259895,true,1,carrier0_Van_1,pickup,shipment_43,None,05:06:00,00:00:00,None
...,...,...,...,...,...,...,...,...,...,...,...
1201,carrier_0,-2997.1683157259895,true,2,carrier0_Van_0,delivery,shipment_121,None,19:58:15,00:01:05,"[439142856_6-439142856_7-439142856_8, 10638314..."
1202,carrier_0,-2997.1683157259895,true,2,carrier0_Van_0,delivery,shipment_134,None,20:00:50,00:01:02,[893949565_0-893949565_1-893949565_2-893949565...
1203,carrier_0,-2997.1683157259895,true,2,carrier0_Van_0,delivery,shipment_15,None,20:03:23,00:01:48,"[458546718_0-458546718_1, 527920067_0-52792006..."
1204,carrier_0,-2997.1683157259895,true,2,carrier0_Van_0,delivery,shipment_193,None,20:06:41,00:03:06,"[897141780_0-897141780_1, 897141780_2-89714178..."
